In [7]:
import sqlite3
import os

# Connexion à la base de données (le fichier .db sera créé si il n'existe pas)
conn = sqlite3.connect('infractions.db')

# Création d'un curseur pour exécuter des requêtes SQL
cursor = conn.cursor()

# Création des tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS infractions (
    code_infr INT PRIMARY KEY,
    lib_infr VARCHAR(50)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS départements (
    code_dep VARCHAR(50) PRIMARY KEY,
    nom_dep VARCHAR(50) NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS force_odre (
    force_odre VARCHAR(50) PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS annee (
    annee VARCHAR(50) PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS périmètres (
    lib_perimetre VARCHAR(50) PRIMARY KEY,
    force_odre VARCHAR(50) NOT NULL,
    FOREIGN KEY(force_odre) REFERENCES force_odre(force_odre)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS brigade (
    id_brigade VARCHAR(50) PRIMARY KEY,
    lib_brigade VARCHAR(50) NOT NULL,
    lib_perimetre VARCHAR(50),
    code_dep VARCHAR(50) NOT NULL,
    force_odre VARCHAR(50) NOT NULL,
    FOREIGN KEY(lib_perimetre) REFERENCES périmètres(lib_perimetre),
    FOREIGN KEY(code_dep) REFERENCES départements(code_dep),
    FOREIGN KEY(force_odre) REFERENCES force_odre(force_odre)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS fait (
    code_infr INT,
    id_brigade VARCHAR(50),
    annee VARCHAR(50),
    nb_infraction INT,
    PRIMARY KEY(code_infr, id_brigade, annee),
    FOREIGN KEY(code_infr) REFERENCES infractions(code_infr),
    FOREIGN KEY(id_brigade) REFERENCES brigade(id_brigade),
    FOREIGN KEY(annee) REFERENCES annee(annee)
)
''')

# Enregistrement des modifications
conn.commit()

# Fermeture de la connexion
conn.close()

print("Base de données 'infractions.db' créée avec succès!")

Base de données 'infractions.db' créée avec succès!


In [8]:
import pandas as pd
import os

# Charger le fichier Excel
file_path = "crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx"
xls = pd.ExcelFile(file_path)

# Créer un dossier pour stocker les fichiers découpés
output_dir = "decoupage_fichiers/"
os.makedirs(output_dir, exist_ok=True)

# Récupérer les noms des feuilles en ignorant la première
sheet_names = xls.sheet_names[1:]

# Parcourir les feuilles restantes et sauvegarder chaque feuille en CSV
for sheet_name in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    output_file = os.path.join(output_dir, f"{sheet_name}.csv")
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Fichier enregistré : {output_file}")

print("Découpage terminé !")


Fichier enregistré : decoupage_fichiers/Services PN 2012.csv
Fichier enregistré : decoupage_fichiers/Services PN 2013.csv
Fichier enregistré : decoupage_fichiers/Services PN 2014.csv
Fichier enregistré : decoupage_fichiers/Services PN 2015.csv
Fichier enregistré : decoupage_fichiers/Services PN 2016.csv
Fichier enregistré : decoupage_fichiers/Services PN 2017.csv
Fichier enregistré : decoupage_fichiers/Services PN 2018.csv
Fichier enregistré : decoupage_fichiers/Services PN 2019.csv
Fichier enregistré : decoupage_fichiers/Services PN 2020.csv
Fichier enregistré : decoupage_fichiers/Services PN 2021.csv
Fichier enregistré : decoupage_fichiers/Services GN 2012.csv
Fichier enregistré : decoupage_fichiers/Services GN 2013.csv
Fichier enregistré : decoupage_fichiers/Services GN 2014.csv
Fichier enregistré : decoupage_fichiers/Services GN 2015.csv
Fichier enregistré : decoupage_fichiers/Services GN 2016.csv
Fichier enregistré : decoupage_fichiers/Services GN 2017.csv
Fichier enregistré : dec

On va créer le dataframe qui servira à alimenter la base

In [18]:
def create_df(file_path):
    # Charger le fichier Excel
    xls = pd.ExcelFile(file_path)
    
    # Prendre uniquement les feuilles commençant par "Services"
    sheets = [sheet for sheet in xls.sheet_names if sheet.startswith("Services")]
    
    # Liste pour stocker les données
    data = []
    
    # Parcourir les feuilles
    for sheet in sheets:
        # Identifier l'année et la direction
        libelle_annee = sheet[-4:]
        if "PN" in sheet:
            nom_organisation = "Police Nationale"
            nom_organisation_court = "PN"
        else:
            nom_organisation = "Gendarmerie Nationale"
            nom_organisation_court = "GN"
        
        # Charger les données
        df = pd.read_excel(xls, sheet_name=sheet, header=None)
        
        # Définir les indices de départ selon PN ou GN
        row_start = 3 if "GN" in sheet else 4
        col_start = 2  # Les données commencent en colonne C
        
        # Extraire les valeurs spécifiques pour les ID
        id_departement = df.iloc[0, col_start:]  # Ligne 1, colonnes à partir de C
        id_perimetre = df.iloc[1, col_start:] if "PN" in sheet else pd.Series([""] * (df.shape[1] - col_start))
        id_cgd_csp = df.iloc[2, col_start:] if "PN" in sheet else df.iloc[1, col_start:]
        
        # Extraire les ID infractions et les nombres de faits
        id_infraction = df.iloc[row_start - 1:, 0].reset_index(drop=True)  # Colonne A
        lib_infraction = df.iloc[row_start - 1:, 1].reset_index(drop=True)  # Colonne B
        nombre_faits = df.iloc[row_start - 1:, col_start:].reset_index(drop=True)  # Table des faits
        
        # Transformer les données en format exploitable
        for col in range(nombre_faits.shape[1]):
            for row in range(nombre_faits.shape[0]):
                data.append({
                    "lib_annee": libelle_annee,
                    "nom_organisation": nom_organisation,
                    "id_departement": id_departement.iloc[col],
                    "lib_perimetre": id_perimetre.iloc[col],
                    "lib_cgd_csp" : id_cgd_csp.iloc[col],
                    "id_organisation" : nom_organisation_court + id_cgd_csp.iloc[col] + id_perimetre.iloc[col],
                    "id_infraction": id_infraction.iloc[row],
                    "lib_infraction": lib_infraction.iloc[row],
                    "nombre_faits": nombre_faits.iloc[row, col]
                })

    # Ajouter un identifiant avec l'index du DataFrame
    data_df = pd.DataFrame(data)
    data_df.reset_index(inplace=True)

    return data_df 

In [19]:
# On crée le DF sur la base du fichier Excel avec le format crée au dessus
file_path = "crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx"
data_df = create_df(file_path)

In [20]:
data_df.head()

,index,lib_annee,type_organisation,id_departement,lib_perimetre,lib_cgd_csp,id_organisation,id_infraction,lib_infraction,nombre_faits
0,0,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,Police NationaleDIDPAF PREVESSIN 01DCPAF,1,Règlements de compte entre malfaireurs,0.0
1,1,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,Police NationaleDIDPAF PREVESSIN 01DCPAF,2,Homicides pour voler et à l'occasion de vols,0.0
2,2,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,Police NationaleDIDPAF PREVESSIN 01DCPAF,3,Homicides pour d'autres motifs,0.0
3,3,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,Police NationaleDIDPAF PREVESSIN 01DCPAF,4,Tentatives d'homicides pour voler et à l'occas...,0.0
4,4,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,Police NationaleDIDPAF PREVESSIN 01DCPAF,5,Tentatives homicides pour d'autres motifs,0.0


In [16]:
data = data_df.to_dict(orient='records')
# Création d'un index distinct pour chaque valeur de id_perimetre, id_service et id_annee
unique_perimetre = {val: idx for idx, val in enumerate(pd.unique([d['lib_perimetre'] for d in data]))}
unique_service = {val: idx for idx, val in enumerate(pd.unique([d['lib_cgd_csp'] for d in data]))}
unique_annee = {val: idx for idx, val in enumerate(pd.unique([d['lib_annee'] for d in data]))}

# Mettre à jour les valeurs dans data avec les indices
for d in data:
    if d['id_departement'].startswith('0'):
        d['id_departement'] = d['id_departement'][1:]
    d['id_perimetre'] = unique_perimetre[d['lib_perimetre']]
    d['id_service'] = unique_service[d['lib_cgd_csp']]
    d['id_annee'] = unique_annee[d['lib_annee']]

# Ajouter un identifiant avec l'index du DataFrame
data_df = pd.DataFrame(data)
data_df.reset_index(inplace=True)
data_df.rename(columns={'index': 'id_fait'}, inplace=True)

C:\Users\yanis\AppData\Local\Temp\ipykernel_32784\142398545.py:3: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_perimetre = {val: idx for idx, val in enumerate(pd.unique([d['lib_perimetre'] for d in data]))}
C:\Users\yanis\AppData\Local\Temp\ipykernel_32784\142398545.py:4: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_service = {val: idx for idx, val in enumerate(pd.unique([d['lib_cgd_csp'] for d in data]))}
C:\Users\yanis\AppData\Local\Temp\ipykernel_32784\142398545.py:5: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_annee = {val: idx for idx, val in enumerate(pd.unique([d['lib_annee'] for d in data]))}


In [17]:
data_df.head()

,id_fait,level_0,id_fait,lib_annee,type_organisation,id_departement,lib_perimetre,lib_cgd_csp,id_organisation,id_infraction,lib_infraction,nombre_faits,id_perimetre,id_service,id_annee
0,0,0,0,2012,Police Nationale,1,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,1,Règlements de compte entre malfaireurs,0.0,0,0,0
1,1,1,1,2012,Police Nationale,1,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,2,Homicides pour voler et à l'occasion de vols,0.0,0,0,0
2,2,2,2,2012,Police Nationale,1,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,3,Homicides pour d'autres motifs,0.0,0,0,0
3,3,3,3,2012,Police Nationale,1,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,4,Tentatives d'homicides pour voler et à l'occas...,0.0,0,0,0
4,4,4,4,2012,Police Nationale,1,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,5,Tentatives homicides pour d'autres motifs,0.0,0,0,0
